In [1]:
import json
import re

In [2]:
types = [u'page', u'track', u'identify'] #1,2,3
events = ['', u'Opened Request Treatment Modal', u'Requested Treatment', u'Requested Callback'] #0,1,2,3

domains = 'com|co.in|com.hk|de||co.uk|co.jp|fr|com.br|it|ru|es|ca|com.mx|co.id|com.tr|com.au|pl|com.sa|nl|com.ar|com.eg|co.th|com.pk|co.za|com.my|be|gr|com.vn|co.ve|com.tw|com.ua|at|se|com.co|ro|ch|pt|com.ph|cl|com.ng|com.sg|com.pe|ae|co.kr|co.hu|ie|dk|no|co.il|fi|cz|co.ma|sk|co.nz|com.kw|lk|bg|com.qa|az|kz|com.do|hr|by|com.ec|lt|iq|co.ke|com.bd|com.om|tn|si|co.cr|com.gt|com.pr|com.sv|lv|com.uy|jo|com.bo|ba|com.cu|rs|com.ly|cm|ee|co.ug|com.bh|com.np|com.gh|dz|lu|com.lb|co.uz|ci|com.py|com.ni|hn|com.et|tt|co.tz|mg|sn|cd|com.kh|am|com.af|ge|mu|com.bn|co.mz|com.jm|com.gi|is|com.pa|md|ps|com.na|mn|com.mt|co.bw|bj|kg|ml|rw|co.zm|bs|ht|la|com.bz|co.zw|as|cat|mk|ne|mw|tg|co.ao|gp|gy|bf|ga|li|co|tm|dj|mv|hk|sc|dm|bi|co.vi|vu|ad|com.vc|com.ag|com.fj|to|cf|fm|tk|gg|ws|vg|im|nu|gm|je|ms|me|co.im|tl|com.ai|gl|co.ls|co.je|st|it.ao|com.by|com.tj|pn|sh|com.sl|nr|sm|cg|co.ck|com.sb|com.cy|so|com.nf|com.ve|com.iq|jp|ac|com.tn|in|td'

patterns = [
    u'^(?P<ref>http[s]*://[www.]*clck.yandex.(ru|ua|kz|com|com.tr|by))',
    u'^(?P<ref>http[s]*://[www.]*yandex.(ru|ua|kz|com|com.tr|by)/clck/)',
    u'^(?P<ref>http[s]*://[www.]*yandex.(ru|ua|kz|com|com.tr|by)/(yandsearch|touchsearch|msearch|search))',
    u'^(?P<ref>http[s]*://[www.]*google.(%s)/(search?|m))'%domains,
    u'^(?P<ref>http[s]*://[www.]*google.(%s)/url)'%domains,
    u'^(?P<ref>http[s]*://[www.]*bing.com/search)'
]
bookimed = ['bookimed.com', 'ru.bookimed.com']
bookimed_pattern = u'^http[s]*://[ru.]*bookimed.com(?P<tail>[\w/=?..&@-]*)'

b_r = re.compile(bookimed_pattern)

list_of_fields = ['action', 'action_after_load', 'country', 'gl', 'hl', 'illness', 'prev', 'q', 'rurl', 'show_popup', 'sl', 'sort', 'sortType', 'source', 'sourceid', 'spam_reason', 'subpage', 'text', 'tl', 'tld', 'u', 'url', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term']

In [3]:
def search_cut(search):
    new_search = ''
    if search:
        try:
            fields = re.findall(r"[/?&]([\w_]*)=([\w@1-9-_{}:]*)", search)
            fields = dict(fields)
            for k,v in fields.iteritems():
                if k in list_of_fields: new_search += '&' +str(k) + '=' + str(v)
        except: print search
    return new_search

In [4]:
def cut_ref(ref_str):
    for pattern in patterns:
        if re.match(pattern,ref_str):
            return re.match(pattern,ref_str).groupdict()['ref']
    if ref_str == '':
        return '/'
    if re.match(b_r, ref_str):
        return re.match(b_r, ref_str).groupdict()['tail']
    return ref_str

In [5]:
def get_type_id(type_str):
    try:
        return types.index(type_str)+1
    except:
        return -1

In [6]:
def get_event_id(event_str):
    try:
        return events.index(event_str)
    except:
        return -1

In [7]:
def cut_the_dict(d):
    #flat
    try:
        d['timestamp']=d['timestamp'].pop('$date')
    except:
        d['timestamp']=d['receivedAt'].pop('$date')
    d.update(d.pop('context'))
    d.update(d.pop('page'))
    #delete
    fields = ['__v','_id','gaClientId','messageId','projectId','receivedAt','sentAt', 'channel']
    for field in fields:
        try:
            d.pop(field)
        except:
            pass
    if d['type']=='page':
        d.pop('properties')
    d.pop('library')
    #ids
    d['type'] = get_type_id(d['type'])
    try:
        d['event'] = get_event_id(d['event'])
    except:
        pass

In [8]:
#create short data
f = open('short_bookimed.json', 'w+')
with open('./bookimed.json') as data_file: 
    for line in data_file:
        try:
            data = json.loads(line[:-1])
            cut_the_dict(data)
            try:
                url = data['url'].split('//')[1].split('/')[0].lower()
            except:
                url = ''
            if url in bookimed:
                del data['url']
                data['referrer']=cut_ref(data['referrer'])
                json.dump(data, f)
                f.write('\n')
        except:
            print json.loads(line[:-1])
f.close()